# Video understanding - shot based chapter analysis
Chapter analysis breaks a video into structured segments, summarizing key audio and visual content for each segment. In the media industry, this helps editors, producers, and analysts quickly understand the storyline, highlight important moments, and streamline content indexing for TV shows, films, documentaries, and news programs.

In this sample notebook, we consume the metadata extracted using the Video Understanding tool, which includes shot summaries and audio transcripts with associated timestamps. We will retrieve this metadata and use it for further chapter analysis with LLMs.

![media_chapter](./statics/video-media-chapter.png)

We will use [Meridian](https://en.wikipedia.org/wiki/Meridian_(film)), an open-source movie, for this analysis.

In [ ]:
from utils import dynamodb_tool, bedrock_tool
import json
import boto3
from IPython.display import JSON, Markdown, display

The results of the video understanding tool are stored in both Amazon S3 and Amazon DynamoDB, and you can access them directly within the same AWS account.

In this lab, we will retrieve the extracted video summary and audio transcription from DynamoDB using the `dynamodb_utils` functions. The tool manages four DynamoDB tables:

- Task table: **bedrock_mm_extr_srv_video_task** — Stores metadata related to video processing tasks, such as task ID, status, creation time, and the original request.
- Frame table: **bedrock_mm_extr_srv_video_frame** — Stores frames associated with task IDs, containing frame-level analysis results generated by the frame-based pipeline.
- Shot table: **bedrock_mm_extr_srv_video_shot** — Stores shot-level extraction results associated with task IDs, populated by the shot-based pipeline.
- Transcript table: **bedrock_mm_extr_srv_video_transcript** — Stores audio transcriptions associated with each task ID.

You can find the task ID in the Video Understanding Tool UI.

Go to the Shot-based section and select the Meridan video that was processed earlier using the UI. Click the Get Task ID link then replace the value below with it.

![Find task Id](./statics/find-task-id.png)


In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Retrieve transcripts from the Video Understanding Tool's managed database.

In [ ]:
transcripts = dynamodb_tool.get_transcripts(task_id)
print(json.dumps(transcripts,indent=2))

Retrieve shot summaries from the Video Understanding Tool's managed database.

In [ ]:
shots = dynamodb_tool.get_shot_outputs(task_id=task_id, output_names=["Summarize shot"])
print(json.dumps(shots,indent=2))

## Directly analyze chapters using both the transcripts and shot summaries
For shorter videos (e.g., under 10 minutes), you can pass the audio transcripts and shot summaries directly to an LLM to analyze chapters.

In this example, we use **Nova Pro**, which provides balanced reasoning capabilities. This model is well-suited for handling large input sizes (for longer videos) and performing reasoning tasks such as aligning timestamps between shot summaries and audio transcripts.

In [ ]:
# To reduce processing time for demo, we only use the first 30 shots as input
shots_subset = shots[0:30] if len(shots) > 30 else shots

prompt = f'''
You are a media expert that summarizes videos into chapter-based segments. 
A chapter is a coherent segment of a video that groups together related content, events, or scenes. Each chapter has a defined start and end time and can include both audio and visual information, providing a summarized view of that portion of the video.
You are given two inputs:
Shot-level visual descriptions with timestamps.
Audio transcription text with timestamps.
Your task is to:
Merge the visual and audio information to identify meaningful chapters.
Group adjacent frames and transcript segments into coherent chapters.
For each chapter, provide:
Start time (earliest timestamp from shot start_time/transcript).
End time (latest timestamp from shot end_time/transcript).
chapter summary (a concise description combining visual and audio context).
Ensure chapter boundaries reflect major changes in visuals, topics, or conversations.
Output the results as in the markdown format.

Shot Summary:
{shots_subset}
Audio Transcript:
{transcripts}
'''

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="us.amazon.nova-pro-v1:0", 
    prompt=prompt, 
    inference_config={
      "maxTokens": 10000,
      "topP": 0.1,
      "temperature": 0.3
    }
)
result = bedrock_tool.parse_converse_response(response)
display(Markdown(result.replace("\\n", "\n")))

## Chatper analysis for long form videos
For longer media, such as a two-hour film with dense shot switches and extensive transcripts, providing all this information may exceed the LLM’s context window, and larger inputs often lead to less accurate results. A good optimization is to first summarize the audio transcripts into chapters, then use these chapter boundaries to group the visual (shot summaries) for a final chapter-level summary.

Deciding between audio-based or vision-based chapters depends on your video content. For most epsodical videos—such as TV shows, films, documentaries, sports, or news—that are professionally edited and where audio carries much of the information, audio-based chapters are generally more suitable as the baseline, with visual information blended in for additional context.

### Summarize the audio transcript into chapters

In [ ]:
prompt = f'''
You are an media expert that summarizes video audio transcripts into coherent, non-overlapping chapters. Your task is to analyze the transcript and segment the video into chapters with clear titles, summaries and time ranges.
Guidelines:
- Chapters must fully cover the video duration without gaps. The first chatper should start from 0 second.
- For time ranges with no audio transcript available, create a default chapter (e.g., "No Audio Transcript") spanning that duration.
- Chapters must be sequential and continuous, with the end time of one chapter matching the start time of the next.
Audio Transcript:
{transcripts}
'''

Tool configuration provides a more reliable way to obtain structured outputs from a Foundation Model. In this example, we define the following tool configuration and send it to the Bedrock Converse API.

In [ ]:
tool_config = {
    "toolChoice": {
        "tool": {
            "name": "audio_chapters"
        }
    },
    "tools": [
        {
            "toolSpec": {
                "name": "audio_chapters",
                "description": "Analyze the input audio transcripts and segment the video into chapters with clear titles, summaries and time ranges.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "chapters": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "title": {
                                            "type": "string"
                                        },
                                        "start_time": {
                                            "type": "number"
                                        },
                                        "end_time": {
                                            "type": "number"
                                        },
                                        "audio_summary": {
                                            "type": "string"
                                        }
                                    },
                                    "required": [
                                        "title",
                                        "start_time",
                                        "end_time",
                                        "audio_summary"
                                    ]
                                }
                            }
                        },
                        "required": [
                            "chapters"
                        ]
                    }
                }
            }
        }
    ]
}

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="us.amazon.nova-pro-v1:0", 
    prompt=prompt, 
    tool_config=tool_config,
    inference_config={
      "maxTokens": 10000,
      "topP": 0.1,
      "temperature": 0.3
    }
)

result = bedrock_tool.parse_converse_response(response)
print(json.dumps(json.loads(result),indent=2))

### Align shots to the audio chatpers 
To generate chapter summaries using both the audio transcripts and the visual (shot) summaries.

In [ ]:
chapters = json.loads(result)
for chapter in chapters.get("chapters"):
    print(f'Processing chatper: {chapter["title"]} [{chapter.get("start_time")}-{chapter.get("end_time")}] s')

    # Find overlappring shots
    overlapping_shots = [
        item for item in shots
        if item["start_time"] < chapter.get("end_time") and item["end_time"] > chapter.get("start_time")
    ]
    overlapping_shots_sorted = sorted(overlapping_shots, key=lambda x: x["start_time"])

    # Generate chapter summary using audio transcript summary and the shot summaries
    chatper_prompt = f'''
    You are a media expert tasked with creating comprehensive chapter summaries for videos. Your job is to combine information from both the audio transcripts (chapter summaries) and the visual content (shot summaries) to produce a unified, coherent chapter summary that reflects the key events and details from the entire video.
    Do not include any Markdown prefixes in the result. Describe the video directly, without starting the summary with leading phrases such as 'The video begins', 'the video opens with'
    Audio transcript summary:
    {chapter.get("audio_summary")}
    Vision shots with summary:
    {overlapping_shots_sorted}
    '''
    response = bedrock_tool.bedrock_converse(
        model_id="us.amazon.nova-pro-v1:0", 
        prompt=chatper_prompt, 
        inference_config={
          "maxTokens": 10000,
          "topP": 0.1,
          "temperature": 0.3
        }
    )
    full_summary = bedrock_tool.parse_converse_response(response)
    chapter["full_summary"] = full_summary

In [ ]:
print(json.dumps(chapters,indent=2))

## Summary
Video chapter analysis is a complex topic, and the methods and logic can vary depending on the video content type and business requirements. This sample demonstrates two representative approaches to analyzing your video based on metadata extracted using the video understanding tool, serving as a starting point. You can further extend and customize the analytics flow to tailor the results to your specific needs.